# Lyric Generator!

Further to my studies of NLP, I decided to see whether I could harness AI to write new lyrics based on my previous listening habits. 

In [6]:
import spotipy.util as util
import spotipy
import os.path as path
import requests
from bs4 import BeautifulSoup

username = 'ninahew'
spotify_client_id ='a758a8092e3b45949388c4123f79948f'
spotify_client_secret = 'df4c710d84b64ab7810f0b9b055a07f2'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'
genius_key = 'sQkNdeBg0aT9rz89yElRfHbxE_UIW3kFs9IkXPIH6Ab4cH8DvfikT09FU56MVjZP'
spotify_playlist_id = 'spotify:playlist:37i9dQZF1EtmAVggpyt82c'
spotify_user_id = 'spotify:user:ninahew'

from spotipy.oauth2 import SpotifyClientCredentials

#spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, 
#                                                                              client_secret=client_secret))

In [7]:

class GetLyrics():
    
    def __init__(self, spotify_client_id, spotify_client_secret, user_id, playlist_id, genius_key):
        self.spotify_client_id = spotify_client_id
        self.spotify_client_secret = spotify_client_secret
        self.user_id = user_id
        self.playlist_id = playlist_id
        self.genius_key = genius_key
        
    def get_playlist_info(self):
        token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()
        sp = spotipy.Spotify(token)
        playlist = sp.user_playlist_tracks(self.user_id, self.playlist_id)
        self.playlist = playlist
        return self.playlist
    
    def get_track_names(self):
        track_names = []
        for song in range(len(self.playlist['items'])):
            track_names.append(self.playlist['items'][song]['track']['name'])
        self.track_names = track_names
        return self.track_names
    
    def get_track_artists(self):
        track_artists = []
        for song in range(len(self.playlist['items'])):
            track_artists.append(self.playlist['items'][song]['track']['artists'][0]['name'])
        self.track_artists = track_artists
        return self.track_artists
        
    def request_song_info(self, track_name, track_artist):
        self.track_name = track_name
        self.track_artist = track_artist
        base_url = 'https://api.genius.com'
        headers = {'Authorization': 'Bearer ' + self.genius_key}
        search_url = base_url + '/search'
        data = {'q': track_name + ' ' + track_artist}
        response = requests.get(search_url, data=data, headers=headers)
        self.response = response
        return self.response

    def check_hits(self):
        json = self.response.json()
        remote_song_info = None
        for hit in json['response']['hits']:
            if self.track_artist.lower() in hit['result']['primary_artist']['name'].lower():
                remote_song_info = hit
                break
        self.remote_song_info = remote_song_info
        return self.remote_song_info
    
    def get_url(self):
        song_url = self.remote_song_info['result']['url']
        self.song_url = song_url
        return self.song_url
    
    def scrape_lyrics(self):
        page = requests.get(self.song_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics1 = html.find("div", class_="lyrics")
        lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
        if lyrics1:
            lyrics = lyrics1.get_text()
        elif lyrics2:
            lyrics = lyrics2.get_text()
        elif lyrics1 == lyrics2 == None:
            lyrics = None
        return lyrics

    def get_lyrics(self):
        playlist = GetLyrics.get_playlist_info(self)
        track_names = GetLyrics.get_track_names(self)
        track_artists = GetLyrics.get_track_artists(self)
        song_lyrics = []
        for i in range(len(self.track_names)):
            print("\n")
            print(f"Working on track {i}.")
            response = GetLyrics.request_song_info(self, self.track_names[i], self.track_artists[i])
            remote_song_info = GetLyrics.check_hits(self)
            if remote_song_info == None:
                lyrics = None
                print(f"Track {i} is not in the Genius database.")
            else:
                url = GetLyrics.get_url(self)
                lyrics = GetLyrics.scrape_lyrics(self)
                if lyrics == None:
                    print(f"Track {i} is not in the Genius database.")
                else:
                    print(f"Retrieved track {i} lyrics!")
            song_lyrics.append(lyrics)
        return song_lyrics


In [9]:
songs = GetLyrics(spotify_client_id, spotify_client_secret, spotify_user_id, spotify_playlist_id, genius_key)
song_lyrics = songs.get_lyrics()

<ipython-input-7-7c333718245f>:11: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()




Working on track 0.
Retrieved track 0 lyrics!


Working on track 1.
Track 1 is not in the Genius database.


Working on track 2.
Track 2 is not in the Genius database.


Working on track 3.
Retrieved track 3 lyrics!


Working on track 4.
Retrieved track 4 lyrics!


Working on track 5.
Retrieved track 5 lyrics!


Working on track 6.
Track 6 is not in the Genius database.


Working on track 7.
Retrieved track 7 lyrics!


Working on track 8.
Track 8 is not in the Genius database.


Working on track 9.
Retrieved track 9 lyrics!


Working on track 10.
Track 10 is not in the Genius database.


Working on track 11.
Retrieved track 11 lyrics!


Working on track 12.
Track 12 is not in the Genius database.


Working on track 13.
Retrieved track 13 lyrics!


Working on track 14.
Track 14 is not in the Genius database.


Working on track 15.
Retrieved track 15 lyrics!


Working on track 16.
Retrieved track 16 lyrics!


Working on track 17.
Track 17 is not in the Genius database.


Working on tra

In [10]:
song_lyrics[0]

"\n\n[Verse 1]\nMirror, mirror on the wall\nDon't say it ’cause I know I'm cute (Ooh, baby)\nLouis down to my drawers\nLV all on my shoes (Ooh, baby)\nI be drippin' so much sauce\nGot a bih lookin’ like RAGÚ (Ooh, baby)\nLit up like a crystal ball\nThat's cool, baby, so is you\nThat's how I roll\n\n[Pre-Chorus]\nIf I'm shinin', everybody gonna shine (Yeah, I'm goals)\nI was born like this, don't even gotta try (Now you know)\nI'm like chardonnay, get better over time (So you know)\nHeard you say I’m not the baddest, bitch, you lie (Haha)\n\n[Chorus]\nIt ain’t my fault that I'm out here gettin’ loose\nGotta blame it on the Goose\nGotta blame it on my juice, baby\nIt ain't my fault that I'm out here makin' news\nI’m the pudding in the proof\nGotta blame it on my juice\nYa-ya-ee, ya-ya-ee, ya-ya-ee, ya-ya-ee\nBlame it on my juice, blame it, blame it on my juice\nYa-ya-ee, ya-ya-ee, ya-ya-ee, ya-ya-ee\nBlame it on my juice, blame it, blame it on my juice (Ooh, baby)\n\n[Verse 2]\nNo, I'm n

In [11]:
type(song_lyrics[0])

str

In [12]:
import re

In [20]:
text = song_lyrics[0]
re.findall('\[.*\]', text)

['[Verse 1]',
 '[Pre-Chorus]',
 '[Chorus]',
 '[Verse 2]',
 '[Pre-Chorus]',
 '[Chorus]',
 '[Bridge]',
 '[Chorus]']

In [33]:
section_labels = [re.findall('\[[^\]]+\]', song) for song in song_lyrics if song is not None]


In [38]:
label_counter = {}

for song in section_labels:
    for label in song:
        if label not in label_counter:
            label_counter[label] = 1
        else:
            label_counter[label] += 1
label_counter

{'[Verse 1]': 34,
 '[Pre-Chorus]': 25,
 '[Chorus]': 83,
 '[Verse 2]': 28,
 '[Bridge]': 11,
 '[unverified]': 1,
 '[Intro]': 11,
 '[Outro]': 9,
 '[Verse I]': 1,
 '[Verse II]': 1,
 '[Spoken Intro]': 1,
 '[Pre-Chorus 1]': 2,
 '[Intro: Timbaland]': 1,
 '[Verse 1: Nelly Furtado & (Timbaland)]': 1,
 '[Chorus: Nelly Furtado]': 3,
 '[Verse 2: Nelly Furtado & (Timbaland)]': 1,
 '[Interlude: Nelly Furtado & (Timbaland)]': 1,
 '[Bridge: Nelly Furtado & (Timbaland)]': 1,
 '[Post-Chorus]': 1,
 '[Interlude]': 4,
 '[Chorus: Bailey & Collins]': 1,
 '[Verse 1: Bailey]': 1,
 '[Bridge: Collins]': 2,
 '[Verse 2: Bailey]': 1,
 '[Instrumental]': 2,
 '[Bridge: Bailey & Collins]': 1,
 '[Verse 1: Cardi B]': 1,
 '[Verse 2: Bruno Mars]': 1,
 '[Pre-Chorus: Bruno Mars]': 2,
 '[Chorus: Bruno Mars]': 3,
 '[Verse 3: Bruno Mars]': 1,
 '[Bridge: Bruno Mars & Cardi B]': 1,
 '[Outro: Cardi B & Bruno Mars, Both]': 1,
 '[Refrain: Britney Spears]': 3,
 '[Verse 1: Britney Spears]': 1,
 '[Verse 2: Britney Spears]': 1,
 '[Bridg

In [73]:
without_labels = [re.sub('\[[^\]]+\]', '', song) for song in song_lyrics if song is not None]
without_newline = [re.sub('\n', ' ', song) for song in without_labels]
final_format = [re.sub('([a-z])([A-Z])', "\g<1> \g<2>", song) for song in without_newline]
final_format

["   Mirror, mirror on the wall Don't say it ’cause I know I'm cute (Ooh, baby) Louis down to my drawers LV all on my shoes (Ooh, baby) I be drippin' so much sauce Got a bih lookin’ like RAGÚ (Ooh, baby) Lit up like a crystal ball That's cool, baby, so is you That's how I roll   If I'm shinin', everybody gonna shine (Yeah, I'm goals) I was born like this, don't even gotta try (Now you know) I'm like chardonnay, get better over time (So you know) Heard you say I’m not the baddest, bitch, you lie (Haha)   It ain’t my fault that I'm out here gettin’ loose Gotta blame it on the Goose Gotta blame it on my juice, baby It ain't my fault that I'm out here makin' news I’m the pudding in the proof Gotta blame it on my juice Ya-ya-ee, ya-ya-ee, ya-ya-ee, ya-ya-ee Blame it on my juice, blame it, blame it on my juice Ya-ya-ee, ya-ya-ee, ya-ya-ee, ya-ya-ee Blame it on my juice, blame it, blame it on my juice (Ooh, baby)   No, I'm not a snack at all Look, baby, I'm the whole damn meal (Ooh, baby) Dav